# Youtube Content Chatbot


In [ ]:
pip install -U langchain-huggingface


  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install youtube-transcript-api

In [22]:
from youtube_transcript_api import YouTubeTranscriptApi
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv

In [23]:
load_dotenv()

True

In [24]:
# Document loader
yt_api = YouTubeTranscriptApi()
video_id='hmtuvNfytjM'
transcript= yt_api.fetch(video_id)

In [25]:
print(transcript)

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text="This is like a crazy amount of power for\xa0\none piece of technology and it's happened\xa0\xa0", start=0.16, duration=3.44), FetchedTranscriptSnippet(text='to us so fast. You just launched GPT-5. A kid\xa0\nborn today will never be smarter than AI. How\xa0\xa0', start=3.6, duration=4.64), FetchedTranscriptSnippet(text="do we figure out what's real and what's not\xa0\nreal? We haven't put a sex bot avatar in ChatGPT\xa0\xa0", start=8.24, duration=4.16), FetchedTranscriptSnippet(text='yet. Super intelligence. What does that\xa0\nactually mean? This thing is remarkable.', start=12.4, duration=7.68), FetchedTranscriptSnippet(text="I'm about to interview Sam Alman, the CEO\xa0\nof Open AI. Open AI. Open AI. Reshaping\xa0\xa0", start=20.08, duration=5.68), FetchedTranscriptSnippet(text="industries. Dude's a straightup tech lord. Let's\xa0\nbe honest. Right now, they're trying to build a\xa0\xa0", start=25.76, duration=4.4), FetchedT

In [26]:
transcript.snippets

[FetchedTranscriptSnippet(text="This is like a crazy amount of power for\xa0\none piece of technology and it's happened\xa0\xa0", start=0.16, duration=3.44),
 FetchedTranscriptSnippet(text='to us so fast. You just launched GPT-5. A kid\xa0\nborn today will never be smarter than AI. How\xa0\xa0', start=3.6, duration=4.64),
 FetchedTranscriptSnippet(text="do we figure out what's real and what's not\xa0\nreal? We haven't put a sex bot avatar in ChatGPT\xa0\xa0", start=8.24, duration=4.16),
 FetchedTranscriptSnippet(text='yet. Super intelligence. What does that\xa0\nactually mean? This thing is remarkable.', start=12.4, duration=7.68),
 FetchedTranscriptSnippet(text="I'm about to interview Sam Alman, the CEO\xa0\nof Open AI. Open AI. Open AI. Reshaping\xa0\xa0", start=20.08, duration=5.68),
 FetchedTranscriptSnippet(text="industries. Dude's a straightup tech lord. Let's\xa0\nbe honest. Right now, they're trying to build a\xa0\xa0", start=25.76, duration=4.4),
 FetchedTranscriptSnippet(text

In [27]:
# [doc.text for doc in transcript]
# we need to convert this text into a proper format
transcript_text = "  ".join(doc.text for doc in transcript)
transcript_text

'This is like a crazy amount of power for\xa0\none piece of technology and it\'s happened\xa0\xa0  to us so fast. You just launched GPT-5. A kid\xa0\nborn today will never be smarter than AI. How\xa0\xa0  do we figure out what\'s real and what\'s not\xa0\nreal? We haven\'t put a sex bot avatar in ChatGPT\xa0\xa0  yet. Super intelligence. What does that\xa0\nactually mean? This thing is remarkable.  I\'m about to interview Sam Alman, the CEO\xa0\nof Open AI. Open AI. Open AI. Reshaping\xa0\xa0  industries. Dude\'s a straightup tech lord. Let\'s\xa0\nbe honest. Right now, they\'re trying to build a\xa0\xa0  super intelligence that could far exceed humans\xa0\nin almost every field. And they just released\xa0\xa0  their most powerful model yet. Just a couple years\xa0\nago, that would have sounded like science fiction.\xa0\xa0  Not anymore. In fact, they\'re not alone. We are\xa0\nin the middle of the highest stakes global race\xa0\xa0  any of us have ever seen. Hundreds of billions of\xa

In [28]:
# Text splitting
splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)


In [29]:
chunks=splitter.split_text(transcript_text)
chunks

["This is like a crazy amount of power for\xa0\none piece of technology and it's happened\xa0\xa0  to us so fast. You just launched GPT-5. A kid\xa0\nborn today will never be smarter than AI. How\xa0\xa0  do we figure out what's real and what's not\xa0\nreal? We haven't put a sex bot avatar in ChatGPT\xa0\xa0  yet. Super intelligence. What does that\xa0\nactually mean? This thing is remarkable.  I'm about to interview Sam Alman, the CEO\xa0\nof Open AI. Open AI. Open AI. Reshaping\xa0\xa0  industries. Dude's a straightup tech lord. Let's",
 "be honest. Right now, they're trying to build a\xa0\xa0  super intelligence that could far exceed humans\xa0\nin almost every field. And they just released\xa0\xa0  their most powerful model yet. Just a couple years\xa0\nago, that would have sounded like science fiction.\xa0\xa0  Not anymore. In fact, they're not alone. We are\xa0\nin the middle of the highest stakes global race\xa0\xa0  any of us have ever seen. Hundreds of billions of",
 "dollars

In [30]:
len(chunks)#total no of chunks created

151

In [31]:
embedding=HuggingFaceEmbeddings(model='sentence-transformers/all-MiniLM-L6-v2')

In [32]:
# Vector-stores : FAISS
vector_store=FAISS.from_texts(chunks,embedding)

In [33]:
vector_store.similarity_search(query='Tell me about AI ?',k=2)

[Document(id='be6dd758-77d6-48a3-b284-f125ee5e1a93', metadata={}, page_content='that? And then philosophically, how are you and\xa0\xa0  others working on building that technology in\xa0\na way that really helps and not hurts people?\xa0\xa0  So just taking the tactical part right now.\xa0\nMy understanding is that there are three big\xa0\xa0  categories that have been limiting factors for\xa0\nAI. The first is compute, the second is data and\xa0\xa0  the third is algorithmic design. How do you think\xa0\nabout each of those three categories right now?\xa0\xa0  And if you were to help someone understand'),
 Document(id='934d18a6-63d1-42d3-b915-8ab48d2cebbf', metadata={}, page_content="never knew the world without AI. So they don't\xa0\xa0  really think about it. It's just this thing that's\xa0\ngoing to be there in everything. and and they will\xa0\xa0  think about like the companies that built on it\xa0\nand what they did with it and the kind of like\xa0\xa0  political leaders the dec

In [34]:
# Retrievers
retriever = vector_store.as_retriever(search_kwargs={"k":4})

In [35]:
retriever.invoke("How to start a new application based company of Artificial Intelligence?")

[Document(id='f1e5a111-699c-4bc9-bc10-a1ab79e95267', metadata={}, page_content='time to go create something totally new, to go\xa0\xa0  invent something, to start a company, whatever\xa0\nit is. I think it is probably possible now to\xa0\xa0  start a company that is a oneperson company that\xa0\nwill go on to be worth like more than a billion\xa0\xa0  dollars and more importantly than that deliver an\xa0\namazing product and service to the world and that\xa0\xa0  that is like a crazy thing. You have access to'),
 Document(id='80fd824d-2882-4181-98f0-a965e8edb7b4', metadata={}, page_content="president or that president did and you know the\xa0\xa0  role of the AI companies is all these companies\xa0\nand people and institutions before us built up\xa0\xa0  this scaffolding we added our one layer on top and\xa0\nnow people get to stand on top of that and add one\xa0\xa0  layer and the next and the next and many more And\xa0\nthat is the beauty of our society. We kind of all  I I love this

In [36]:
# Augmentation
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""You are a helpful assistant.
Answer ONLY from the following context.
If the context is insufficient, say: I don't know.

Context:
{context}

Question:
{question}
""",
    input_variables=["context", "question"]
)

In [64]:
question="What do you know about Super Intelligence?"

In [65]:
context=retriever.invoke(question)

In [66]:
context

[Document(id='e4c8a581-a43e-4f60-b00a-350f3dda61f2', metadata={}, page_content="people that were actually in labs and and\xa0\xa0  trying to build what we have defined as super\xa0\nintelligence. And it did seem like there were\xa0\xa0  these two camps forming. There's a group of\xa0\npeople who are using the tools like you in this\xa0\xa0  conversation and building tools for others\xa0\nsaying this is going to be a really useful\xa0\xa0  future that we're all moving toward. Your life is\xa0\ngoing to be full of choice and we've talked about\xa0\xa0  our my potential kids and and their futures."),
 Document(id='a8c3a69c-17b9-49eb-8093-ed0ab8201aa4', metadata={}, page_content='that point and play it out a little bit. Nothing\xa0\xa0  further than that. Like at what point would one\xa0\nof these systems come up with general relativity?\xa0\xa0  Interesting question is did you like if we think\xa0\nabout that forward like like if we think of where\xa0\xa0  we are now should a if if we nev

In [67]:
context = " ".join(i.page_content for i in context)

In [68]:
final_prompt = prompt.format(question=question, context=context)
print(final_prompt)


You are a helpful assistant.
Answer ONLY from the following context.
If the context is insufficient, say: I don't know.

Context:
people that were actually in labs and and    trying to build what we have defined as super 
intelligence. And it did seem like there were    these two camps forming. There's a group of 
people who are using the tools like you in this    conversation and building tools for others 
saying this is going to be a really useful    future that we're all moving toward. Your life is 
going to be full of choice and we've talked about    our my potential kids and and their futures. that point and play it out a little bit. Nothing    further than that. Like at what point would one 
of these systems come up with general relativity?    Interesting question is did you like if we think 
about that forward like like if we think of where    we are now should a if if we never got another 
piece of physics data. Yeah. Do we expect that a    really good super intelligence could 

In [ ]:
# Generation part : connect this final_prompt with LLM 



In [2]:
!pip install requests

In [ ]:
import requests

url = "http://127.0.0.1:11434/v1/chat/completions"

data = {
    "model": "mistral:7b",
    "messages": [
        {"role": "user", "content": final_prompt}
    ],
    "max_tokens": 1000,
    "stream": False
}

resp = requests.post(url, json=data)
answer=resp.json()["choices"][0]["message"]["content"]
print(answer)

According to our conversation, Super Intelligence refers to an artificial intelligence system (similar in capabilities and potential impact as a human superintelligence). It suggests that such entities are now being actively developed by various individuals and organizations. These systems aim to build tools for others and serve as agents capable of making decisions independently or performing research, similar to humans doing their own work.
